1) where the hotspots are currently and what areas to focus on next on a map.

2) How would you calculate the fatality rate and what issues do you see with presenting those figures based on this dataset?

3) Please cite any additional sources you used for research and reference any existing dashboards you used for inspiration
 

In [2]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta

In [5]:
src_path = {'wc':'time_series_covid19_confirmed_global.csv','wd':'time_series_covid19_deaths_global.csv','uc':'time_series_covid19_confirmed_US.csv','ud':'time_series_covid19_deaths_US.csv','geo':'UID_ISO_FIPS_LookUp_Table.csv'}

target_path = {'summary':'time_series_covid19.csv'}

In [6]:
def get_startdate(data,gaps=5):
    # Get last available date to applicable form
    start_dt = datetime.strptime(max(data['Date']),'%m/%d/%y').date()
    end_dt = date.today()

    if start_dt < end_dt-timedelta(days=1):
        start_dt -= timedelta(days=gaps)

    # %- option only works for Unix-based systems
    return start_dt.strftime('%-m/%-d/%y')

In [7]:
def write_file(data,type,date,target_path=target_path):
    if type == 'new':
        filename = 'csse_covid_19_time_series/'+target_path['summary']
    elif type == 'remote':
        filename = 'csse_covid_19_time_series/'+target_path['summary']
    else:
        filename = 'csse_covid_19_time_series/'+target_path['summary'][:-4]+'_'+date[:-2].replace('-','')+'_old.csv'
    print('{}: {}'.format(type,filename))
    data.to_csv(filename,index=False)

In [8]:
def reorder_columns(cols):
    # adm0_a3
    # Province/State
    # Country/Region
    # Lat
    # Long
    # Population
    lst = cols
    bases_idx = [cols.get_loc('adm0_a3'),cols.get_loc('Province/State'),cols.get_loc('Country/Region'),cols.get_loc('Lat'),cols.get_loc('Long'),cols.get_loc('Population')]
    bases = cols[bases_idx]

    return list(bases) + list(lst.drop(bases))

In [9]:
def clean_data(wc, wd, uc, ud, geo, cutoff=60):
    # Get least recent date (reporting time gaps)
    min_latest = datetime.strptime(min(wc.columns[-1],wd.columns[-1],uc.columns[-1],ud.columns[-1]),'%m/%d/%y')
    max_first = datetime.strptime(max(wc.columns[2],wd.columns[2],uc.columns[6],ud.columns[6]),'%m/%d/%y')
    min_date = max(min_latest-timedelta(days=cutoff),max_first)

    start_date = min_date.strftime('%-m/%-d/%y')
    end_date = min_latest.strftime('%-m/%-d/%y')

    col_list = {'wc':list(wc.columns[:2])+list(wc.columns[wc.columns.get_loc(start_date):wc.columns.get_loc(end_date)+1]),'wd':list(wd.columns[:2])+list(wd.columns[wd.columns.get_loc(start_date):wc.columns.get_loc(end_date)+1]),'uc':list(uc.columns[:6])+list(uc.columns[uc.columns.get_loc(start_date):uc.columns.get_loc(end_date)+1]),'ud':list(ud.columns[:6])+list(ud.columns[ud.columns.get_loc(start_date):ud.columns.get_loc(end_date)+1])
    }

    wc = wc[col_list['wc']].merge(geo,on=['Country/Region','Province/State'])
    wc.drop(wc[((wc['Lat']==0) & (wc['Long']==0) & (wc['Province/State'] != 'Unknown') & (wc['Province/State'].str.contains('Qurantine')!=True)) | (wc['adm0_a3'].isin(['USA','KOR'])) | (wc['adm0_a3'].isnull())].index,inplace=True)
    wc = wc[reorder_columns(wc.columns)]

    wd = wd[col_list['wd']].merge(geo,on=['Country/Region','Province/State'])
    wd.drop(wd[((wd['Lat']==0) & (wd['Long']==0) & (wd['Province/State'] != 'Unknown') & (wd['Province/State'].str.contains('Qurantine')!=True)) | (wd['adm0_a3'].isin(['USA','KOR'])) | (wd['adm0_a3'].isnull())].index,inplace=True)
    wd = wd[reorder_columns(wd.columns)]

    uc = uc.loc[((uc['Lat']!=0) | (uc['Long']!=0)) & ((uc['adm0_a3'].notnull())),col_list['uc']]
    us_geo = uc.groupby(['adm0_a3','Country/Region','Province/State'])[['Lat','Long']].mean()
    uc_core = uc.groupby(['adm0_a3','Country/Region','Province/State']).sum().drop(['Lat','Long'],axis=1)
    uc = pd.merge(uc_core,us_geo,on=['adm0_a3','Country/Region','Province/State']).reset_index()
    uc = uc[reorder_columns(uc.columns)]

    ud = ud.loc[((ud['Lat']!=0) | (ud['Long']!=0)) & ((ud['adm0_a3'].notnull())),col_list['ud']]
    ud_core = ud.groupby(['adm0_a3','Country/Region','Province/State']).sum().drop(['Lat','Long'],axis=1)
    ud = pd.merge(ud_core,us_geo,on=['adm0_a3','Country/Region','Province/State']).reset_index()
    ud = ud[reorder_columns(ud.columns)]

    return wc, wd, uc, ud

In [10]:
def group_by_type(data_1, data_2, type):
    # Confirmed and Deaths
    data = pd.concat([data_1,data_2])

    # Population manipulation (No country-level statistics for multi-states)
    base_pop = 1e5
    data['num_states'] = data.groupby(['adm0_a3','Country/Region']).adm0_a3.transform('size')
    data.loc[(data['Province/State'].isnull()) & (data['num_states']>1),'Population'] = 0
    data.drop('num_states',axis=1,inplace=True)

    # Wide to long form transformation
    data = data.melt(id_vars=['adm0_a3','Province/State','Country/Region','Lat','Long','Population'])
    data.rename(columns={'variable':'Date','value':type},inplace=True)
    # data['Date'] = pd.to_datetime(data['Date'],format='%m/%d/%y')
    data[type].fillna(0,inplace=True)
    data['Province/State'].replace(np.nan,'NA',inplace=True)
    # Reorder columns
    data = data[[data.columns[-2]]+list(data.columns[:-2])+[data.columns[-1]]]

    # per 100K (r_: Rate)
    data['r_'+type] = data[type]/data['Population']*base_pop

    # Daily changes (i_: Daily Raw, ri_: Daily Rate)
    data['i_'+type] = data[type]-data.groupby(['adm0_a3','Province/State','Country/Region'])[type].shift(1)
    data['i_'+type].fillna(data[type],inplace=True)
    # print(data[['Date','Country/Region','Province/State',type,'i_'+type]])
    # print(data)
    data['ri_'+type] = data['i_'+type]/data['Population']*base_pop
    # data['ri'+type].fillna(data['r'+type],inplace=True)

    # Country-level grouping
    data['Tot_'+type] = data.groupby(['Date','adm0_a3','Country/Region'])[type].transform('sum')

    data['iTot_'+type] = data.groupby(['Date','adm0_a3','Country/Region'])['i_'+type].transform('sum')
    data['rTot_'+type] = data['Tot_'+type]/data.groupby(['Date','adm0_a3','Country/Region'])['Population'].transform('sum')*base_pop

    data['riTot_'+type] = data['iTot_'+type]/data.groupby(['Date','adm0_a3','Country/Region'])['Population'].transform('sum')*base_pop

    # print(data[['Date','Country/Region','Province/State',type,'i_'+type]])

    data.drop(data[data['Date']==data['Date'][0]].index,inplace=True)
    data = data.round(4)
    data.reset_index(drop=True,inplace=True)
    print(data[['Date','Country/Region','Province/State',type,'i_'+type]])

    # print(data)

    return data

In [11]:
def read_source(start_date=None):
    # Read global data
    wc = pd.read_csv("csse_covid_19_time_series/"+src_path['wc'])
    wd = pd.read_csv("csse_covid_19_time_series/"+src_path['wd'])
    # Geographical information will be merged
    wc.drop(['Lat','Long'],axis=1,inplace=True)
    wd.drop(['Lat','Long'],axis=1,inplace=True)

    # Read US data
    uc = pd.read_csv("csse_covid_19_time_series/"+src_path['uc'])
    ud = pd.read_csv("csse_covid_19_time_series/"+src_path['ud'])
    uc.rename(columns={'iso3':'adm0_a3','Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)
    ud.rename(columns={'iso3':'adm0_a3','Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)

    # County-level data to be purged
    uc.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)
    ud.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)

    # Append Population column to infections data
    uc = pd.concat([uc.iloc[:,:5],ud['Population'],uc.iloc[:,5:]],axis=1)

    # Read geographical information
    geo = pd.read_csv(src_path['geo'])
    geo.rename(columns={'iso3':'adm0_a3','Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)
    geo.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)

    # Clean and group datasets
    wc, wd, uc, ud = clean_data(wc, wd, uc, ud, geo)
    confirmed = group_by_type(wc,uc,'Confirmed')
    deaths = group_by_type(wd,ud,'Deaths')

    covid = confirmed.merge(deaths,on=['Date','adm0_a3','Province/State','Country/Region','Lat','Long','Population'])

    # print(covid)

    return covid



In [12]:
def read_appendfile():
    # Read current file
    file_list = target_path['summary']

    read_columns = ['Date','adm0_a3','Province/State','Country/Region','Lat','Long','Population','Confirmed','r_Confirmed','i_Confirmed','ri_Confirmed','Tot_Confirmed','iTot_Confirmed','rTot_Confirmed','riTot_Confirmed','Deaths','r_Deaths','i_Deaths','ri_Deaths','Tot_Deaths','iTot_Deaths','rTot_Deaths','riTot_Deaths']

    try:
        print("Reading previous summary data with the filename: "+file_list)
        last = pd.read_csv("data/"+target_path['summary'])
        new = last.copy()
        if len(read_columns) == len(last.columns):
            status = 'Replace'
        else:
            status = "New"

    except FileNotFoundError:
        print('Current summary file not found\nNo backup file(s) will be created')
        status = "New"


    ds = read_source()
    start_date = get_startdate(ds)
    new = ds

    # else:
    #     # ds = read_source(start_date)
    #     new.drop(new.loc[new['Date']>=start_date,:].index,inplace=True)

    write_date = datetime.strptime(ds.iloc[-1]['Date'],'%m/%d/%y').strftime('%m-%d-%Y')

    # Write to files
    print("Updating summary data: {}".format(status))
    if status == 'Replace':
        write_file(last,'last',write_date)
        write_file(new,'new',write_date)
        write_file(new,'remote',write_date)
    else:
        write_file(new,'new',write_date)
        write_file(new,'remote',write_date)
    print("Summary data successfully updated")

In [13]:
read_appendfile()

Reading previous summary data with the filename: time_series_covid19.csv
Current summary file not found
No backup file(s) will be created
         Date Country/Region  Province/State  Confirmed  i_Confirmed
0      2/7/22    Afghanistan              NA     166924        733.0
1      2/7/22        Albania              NA     264875        251.0
2      2/7/22        Algeria              NA     258478        502.0
3      2/7/22        Andorra              NA      36808        209.0
4      2/7/22         Angola              NA      98409         45.0
...       ...            ...             ...        ...          ...
19975  4/7/22             US      Washington    1461129          0.0
19976  4/7/22             US   West Virginia     498322         87.0
19977  4/7/22             US       Wisconsin    1588689        693.0
19978  4/7/22             US         Wyoming     156264          0.0
19979  4/7/22             US  Virgin Islands      15740         14.0

[19980 rows x 5 columns]
        

In [18]:
covid = pd.read_csv('csse_covid_19_time_series/time_series_covid19.csv')

In [15]:
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [16]:
country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Cases_28_Days,Deaths_28_Days
0,Afghanistan,2022-04-13 01:21:02,33.93911,67.709953,178257,7676,NaN,NaN,457.910600,NaN,NaN,4.306142,4,AFG,1732,30
1,Albania,2022-04-13 01:21:02,41.15330,20.168300,274272,3494,NaN,NaN,9530.613663,NaN,NaN,1.273918,8,ALB,1530,8
2,Algeria,2022-04-13 01:21:02,28.03390,1.659600,265730,6874,NaN,NaN,605.983306,NaN,NaN,2.586836,12,DZA,270,8
3,Andorra,2022-04-13 01:21:02,42.50630,1.521800,40328,153,NaN,NaN,52194.395910,NaN,NaN,0.379389,20,AND,1534,1
4,Angola,2022-04-13 01:21:02,-11.20270,17.873900,99194,1900,NaN,NaN,301.810963,NaN,NaN,1.915438,24,AGO,263,0


In [20]:
# sorted_states_df = covid[covid['adm0_a3']=='USA'].groupby(''.sort_values('Confirmed', ascending= False) 

In [16]:
us_confirmed_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_confirmed_us.csv')
us_deaths_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_deaths_us.csv')

In [5]:
confirmed_per_state = us_confirmed_df.groupby(['Province_State']).sum().drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1).reset_index().sort_values(by='4/7/22', ascending=False)

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/29/22,3/30/22,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22
5,California,0,0,0,0,2,2,2,2,2,...,9097678,9099480,9101621,9109754,9109754,9109754,9112044,9119237,9120316,9121363
49,Texas,0,0,0,0,0,0,0,0,0,...,6732766,6735219,6738919,6741563,6744442,6745371,6736308,6738689,6741250,6745219
11,Florida,0,0,0,0,0,0,0,0,0,...,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004


In [11]:
confirmed_per_state.columns[-1]

'4/7/22'

In [57]:
deaths_per_state = us_deaths_df.groupby(['Province_State']).sum().drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1).reset_index().sort_values(by='4/7/22', ascending=False)

In [58]:
deaths_per_state['rate'] = ((deaths_per_state['4/7/22']/deaths_per_state['Population'])*100).round(2)

In [59]:
deaths_per_state.sort_values(by = 'rate', ascending=False).iloc[1:,:].head()

,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/30/22,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,rate
28,Mississippi,2976149,0,0,0,0,0,0,0,0,...,12396,12397,12399,12399,12399,12402,12407,12410,12410,0.42
3,Arizona,7278717,0,0,0,0,0,0,0,0,...,29268,29268,29268,29268,29268,29268,29268,29681,29681,0.41
41,Oklahoma,3956971,0,0,0,0,0,0,0,0,...,15383,15520,15520,15520,15520,15520,15520,15520,15640,0.40
0,Alabama,4903185,0,0,0,0,0,0,0,0,...,19284,19290,19294,19313,19313,19334,19362,19374,19374,0.40
55,West Virginia,1792147,0,0,0,0,0,0,0,0,...,6823,6831,6835,6835,6835,6839,6744,6741,6749,0.38


In [60]:
deaths_per_state.sort_values(by = '4/7/22', ascending=False).iloc[:,:].head()

,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/30/22,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,rate
5,California,39512223,0,0,0,0,0,0,0,0,...,88879,88950,89012,89012,89012,89052,89154,89221,89247,0.23
49,Texas,28995881,0,0,0,0,0,0,0,0,...,87355,87393,87426,87459,87470,87480,87509,87556,87594,0.30
11,Florida,21477737,0,0,0,0,0,0,0,0,...,73138,73138,73282,73290,73290,73409,73411,73456,73496,0.34
36,New York,19453561,0,0,0,0,0,0,0,0,...,67960,67967,67983,67987,67994,68007,68015,68023,68032,0.35
43,Pennsylvania,12801989,0,0,0,0,0,0,0,0,...,44260,44275,44295,44313,44313,44316,44348,44370,44391,0.35


In [61]:
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
sorted_country_df = country_df[country_df['Confirmed'] > 0].sort_values('Confirmed', ascending= True)
sorted_country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Cases_28_Days,Deaths_28_Days
117,Micronesia,2022-04-13 03:21:08,7.4256,150.5508,1,0,NaN,NaN,0.878619,NaN,NaN,0.000000,583,FSM,0,0
113,Marshall Islands,2022-04-13 03:21:08,7.1315,171.1845,7,0,NaN,NaN,11.983634,NaN,NaN,0.000000,584,MHL,0,0
106,MS Zaandam,2022-04-13 03:21:08,NaN,NaN,9,2,NaN,NaN,NaN,NaN,NaN,22.222222,8888,NaN,0,0
5,Antarctica,2022-04-13 03:21:08,-71.9499,23.3470,11,0,NaN,NaN,NaN,NaN,NaN,0.000000,10,ATA,0,0
76,Holy See,2022-04-13 03:21:08,41.9029,12.4534,29,0,NaN,NaN,3584.672435,NaN,NaN,0.000000,336,VAT,0,0


In [62]:
confirmed_per_state['Confirmed'] = confirmed_per_state['4/7/22']
confirmed_per_state.sort_values('Confirmed', ascending= False)
confirmed_per_state.head()

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/30/22,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,Confirmed
5,California,0,0,0,0,2,2,2,2,2,...,9099480,9101621,9109754,9109754,9109754,9112044,9119237,9120316,9121363,9121363
49,Texas,0,0,0,0,0,0,0,0,0,...,6735219,6738919,6741563,6744442,6745371,6736308,6738689,6741250,6745219,6745219
11,Florida,0,0,0,0,0,0,0,0,0,...,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004,5885004
36,New York,0,0,0,0,0,0,0,0,0,...,4990971,4995734,4999568,5002854,5005818,5010041,5012350,5016872,5022921,5022921
17,Illinois,0,0,1,1,1,1,1,1,1,...,3066645,3068237,3069650,3069650,3069650,3072636,3074435,3076628,3078124,3078124


In [63]:
deaths_per_state = us_deaths_df.groupby(['Province_State']).sum().drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1).reset_index().sort_values(by='4/7/22', ascending=False)
deaths_per_state['Deaths'] = deaths_per_state['4/7/22']
deaths_per_state.sort_values('Deaths', ascending= False).head()

,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/30/22,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,Deaths
5,California,39512223,0,0,0,0,0,0,0,0,...,88879,88950,89012,89012,89012,89052,89154,89221,89247,89247
49,Texas,28995881,0,0,0,0,0,0,0,0,...,87355,87393,87426,87459,87470,87480,87509,87556,87594,87594
11,Florida,21477737,0,0,0,0,0,0,0,0,...,73138,73138,73282,73290,73290,73409,73411,73456,73496,73496
36,New York,19453561,0,0,0,0,0,0,0,0,...,67960,67967,67983,67987,67994,68007,68015,68023,68032,68032
43,Pennsylvania,12801989,0,0,0,0,0,0,0,0,...,44260,44275,44295,44313,44313,44316,44348,44370,44391,44391


In [65]:
deaths_per_state['Fatality_rate'] = (deaths_per_state['4/7/22']/deaths_per_state['Population'])*100
deaths_per_state = deaths_per_state.sort_values('Fatality_rate', ascending= False).iloc[1:]
deaths_per_state.sort_values('Fatality_rate', ascending= False).head()

,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/31/22,4/1/22,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,Deaths,Fatality_rate
28,Mississippi,2976149,0,0,0,0,0,0,0,0,...,12397,12399,12399,12399,12402,12407,12410,12410,12410,0.416982
3,Arizona,7278717,0,0,0,0,0,0,0,0,...,29268,29268,29268,29268,29268,29268,29681,29681,29681,0.407778
41,Oklahoma,3956971,0,0,0,0,0,0,0,0,...,15520,15520,15520,15520,15520,15520,15520,15640,15640,0.395252
0,Alabama,4903185,0,0,0,0,0,0,0,0,...,19290,19294,19313,19313,19334,19362,19374,19374,19374,0.395131
48,Tennessee,6829174,0,0,0,0,0,0,0,0,...,25599,25611,25623,25623,25623,25671,25796,25877,25877,0.378918


In [79]:
us_confirmed_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_confirmed_us.csv')
us_deaths_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_deaths_us.csv')


confirmed_per_state = us_confirmed_df.groupby(['Province_State']).sum().drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1).reset_index().sort_values(by='4/7/22', ascending=False)
confirmed_per_state['Confirmed'] = confirmed_per_state['4/7/22']

deaths_per_state = us_deaths_df.groupby(['Province_State']).sum().drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1).reset_index().sort_values(by='4/7/22', ascending=False)
deaths_per_state = deaths_per_state[deaths_per_state['Population']>0]
deaths_per_state['Deaths'] = deaths_per_state['4/7/22']
deaths_per_state['Fatality_rate'] = (deaths_per_state['4/7/22']/deaths_per_state['Population'])*100

df = pd.merge(confirmed_per_state[{'Province_State', 'Confirmed'}],\
         deaths_per_state[{'Province_State', 'Deaths', 'Fatality_rate'}], on='Province_State')

In [88]:
df.sort_values('Fatality_rate', ascending= False)

,Confirmed,Province_State,Fatality_rate,Deaths
29,795613,Mississippi,0.416982,12410
12,2014020,Arizona,0.407778,29681
25,1035301,Oklahoma,0.395252,15640
23,1296558,Alabama,0.395131,19374
11,2021816,Tennessee,0.378918,25877
36,498322,West Virginia,0.376587,6749
10,2210227,New Jersey,0.374784,33289
28,833485,Arkansas,0.374478,11301
24,1169238,Louisiana,0.369451,17175
9,2393265,Michigan,0.358231,35776


In [81]:
deaths_per_state.shape

(56, 811)

In [84]:
df.shape

(56, 4)